# Importing Required Libararies and Packages

In [1]:
import warnings
warnings.filterwarnings('ignore')

import os

import numpy as np, pandas as pd

import cv2
from deepface import DeepFace

2023-09-10 23:17:06.812750: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-10 23:17:07.393248: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


# Function Definitions

In [2]:
"""
    Modifiying the predefined method of the app class in InsightFace:FaceAnalysis Class, to have a label paramters to set on the image detection.
"""

def custom_draw_on(img, faces, label_lst):
    """
        Function used to draw rectangular box around the detected faces with their labels.
    """
    dimg = img.copy()
    for i in range(len(faces)):
        face = faces[i]
        box = face['facial_area']
        color = (0, 0, 255)
        cv2.rectangle(dimg, (box['x'], box['y']), (box['x']+box['w'], box['y']+box['h']), color, 2)
        cv2.putText(dimg, label_lst[i], (box['x']-1, box['y']-4),cv2.FONT_HERSHEY_COMPLEX,0.7,(0,255,0),1)

    return dimg

def load_label_csv(label_path)->pd.DataFrame:
    """
        Function used to load csv label file
    """
    # Load Label CSV file
    # Step: Searching for a csv file locatted in this directory
    for file in os.listdir(label_path):
        if file.endswith(".csv"): 
            csv_path = os.path.join(label_path, file)

    # Step: load the csv label file
    return pd.read_csv(csv_path)

# Face Detection

In [3]:
face_recognition_model = 'ArcFace'
face_detector_model = 'retinaface'

# Single Image Recognition

In [5]:
test_images_directory = '../data/test/'
label_path = '../data/label'
test_img = '001_5992faf7.jpg'

# Load Label CSV file
df_label = load_label_csv(label_path=label_path)

detected_faces = DeepFace.extract_faces(
    img_path=test_images_directory+test_img,
    detector_backend=face_detector_model
)

found_faces = DeepFace.find(img_path=test_images_directory+test_img,
            db_path=label_path,
            model_name=face_recognition_model,
            detector_backend=face_detector_model,
            enforce_detection=True)


label_lst = []

for found_face in found_faces:

    if len(found_face) != 0:
        # Face was detected
        img_label = 'UnKown'
        
        # checking the ArcFace_cosine threshold
        if found_face['ArcFace_cosine'].values[0]> 0.25:
            img_label_path = found_face['identity'].values[0]
            img_label = df_label[df_label.image_name == img_label_path[img_label_path.rfind('/') + 1:]]['label'].values[0]
            
        label_lst.append(img_label)

    else:
        label_lst.append('Unkown')

img = cv2.imread(test_images_directory+test_img)

rimg= custom_draw_on(img, detected_faces, label_lst)


# Output the labeled Image
cv2.imwrite(f"../report/plots/{test_img}", rimg)

print(f"Recognitioned Faces in {test_img} are: {label_lst}. Exported image with detected faces at: ../report/plots/{test_img}")

Finding representations: 100%|██████████| 6/6 [00:08<00:00,  1.47s/it]


Representations stored in ../data/label/representations_arcface.pkl file.Please delete this file when you add new identities in your database.
find function lasts  10.288462400436401  seconds
Recognitioned Faces in 001_5992faf7.jpg are: ['Kate Winslet']. Exported image with detected faces at: ../report/plots/001_5992faf7.jpg


# List of Images Recognition

In [46]:
test_images_directory = '../data/test/'
label_path = '../data/label'

# Load Label CSV file
# Load Label CSV file
df_label = load_label_csv(label_path=label_path)

# Looping over each image in test image file
for test_image in os.listdir(test_images_directory):
        
    detected_faces = DeepFace.extract_faces(
        img_path=test_images_directory+test_image,
        detector_backend=face_detector_model
    )

    found_faces = DeepFace.find(img_path=test_images_directory+test_image,
                db_path=label_path,
                model_name=face_recognition_model,
                detector_backend=face_detector_model,
                enforce_detection=True)


    label_lst = []

    for found_face in found_faces:

        if len(found_face) != 0:
            # Face was detected
            img_label = 'UnKown'
            
            # checking the ArcFace_cosine threshold
            if found_face['ArcFace_cosine'].values[0]> 0.25:
                img_label_path = found_face['identity'].values[0]
                img_label = df_label[df_label.image_name == img_label_path[img_label_path.rfind('/') + 1:]]['label'].values[0]
                
            label_lst.append(img_label)

        else:
            label_lst.append('Unkown')

    img = cv2.imread(test_images_directory+test_image)

    rimg= custom_draw_on(img, detected_faces, label_lst)

    # Output the labeled Image
    cv2.imwrite(f"../report/plots/{test_image}", rimg)

    print(f"Recognitioned Faces in {test_image} are: {label_lst}. Exported image with detected faces at: ../report/plots/{test_image}")

There are  8  representations found in  representations_arcface.pkl
find function lasts  1.6630043983459473  seconds
Recognitioned Faces in 091_c3ad83af.jpg are: ['Johnny Depp']. Exported image with detected faces at: ../report/plots/091_c3ad83af.jpg
There are  8  representations found in  representations_arcface.pkl
find function lasts  1.4213533401489258  seconds
Recognitioned Faces in 059_aef6a0a8.jpg are: ['UnKown']. Exported image with detected faces at: ../report/plots/059_aef6a0a8.jpg
There are  8  representations found in  representations_arcface.pkl
find function lasts  1.7649304866790771  seconds
Recognitioned Faces in 006_9135205d.jpg are: ['Angelina Jolie', 'UnKown']. Exported image with detected faces at: ../report/plots/006_9135205d.jpg
There are  8  representations found in  representations_arcface.pkl
find function lasts  1.6383800506591797  seconds
Recognitioned Faces in 041_cc0957bf.jpg are: ['Brad Pitt']. Exported image with detected faces at: ../report/plots/041_cc0